# Random Number Generators

For any Monte Carlo (MC) computational technique, random numbers (or pseudo-random numbers) are critical, not only in the sampling characterstics of the generator but also the speed of the sampling. For example, simulating a Large Hadron Collider (LHC) collission can require upwards of $2.5 \times 10^6$ random number calls.

This worksheet is largely based on chapter 7 of [Numerical Recipes](https://numerical.recipes/book.html), and associated primary papers. The following is a list of references for further reading.

*  Hammersley, J.M. and Handscomb, D.C. 1964, Monte Carlo Methods (London: Methuen)
* Kalos, M.H. and Whitlock, P.A. 1986, Monte Carlo Methods (New York: Wiley)
* Bratley, P., Fox, B.L., and Schrage, E.L. 1983, A Guide to Simulation, 2nd ed. (New York: Springer)
* Lepage, G.P. 1978, A New Algorithm for Adaptive Multidimensional Integration, Journal of Computational Physics, vol. 27, pp. 192-203
* Lepage, G.P. 1980, VEGAS: An Adaptive Multidimensional Integration Program, Publication CLNS-80/447, Cornell University
* Numerical Recipes Software 2007, [Complete VEGAS Code Listing](http://numerical.recipes/webnotes?9), Numerical Recipes Webnote No. 9
* Press, W.H., and Farrar, G.R. 1990, Recursive Stratified Sampling for Multidimensional Monte Carlo Integration, Computers in Physics, vol. 4, pp. 190-195
* Numerical Recipes Software 2007, [Complete Miser Code Listing](http://numerical.recipes/webnotes?10), Numerical Recipes Webnote No. 10

## Requirements

This notebook requires a few external dependencies which are imported here. First, it is useful to have interactive plotting via `matplotlib`. To enable this, we need to install the `ipympl` module and restart the kernel.

In [ ]:
# Install `ipympl`.
#!pip install -q ipympl
# get_ipython().kernel.do_shutdown(restart=True)

Next, we configure `matplotlib` and allow widgets in Colab.

In [ ]:
# Allow for interactive plots.
%matplotlib widget
from matplotlib import pyplot as plt

# Allow for widgets in Colab.
try:
    from google.colab import output

    output.enable_custom_widget_manager()
except:
    pass

Finally, we import both `numpy` and `scipy` for matrix operations which are used in the [XORshift RNGs](#scrollTo=XORshift_RNGs) section. We also need `sys` to determine the size of types when performing bit shifts.

In [ ]:
import numpy as np
import scipy as sp
import sys, math, inspect

## Introduction

This tutorial will give an introduction to Random Number Generators (or RNGs, for short). RNG algorithms are really pseudo-random (pRNG). This means they are deterministic. Randomness (Jaynes, [Probability Theory: The Logic of Science](http://www.med.mcgill.ca/epidemiology/hanley/bios601/GaussianModel/JaynesProbabilityTheory.pdf)) is a statement about knowledge, not about causes. pRNGs have causes, but you might not know them, or you might use them in a way so that they appear to be random, or random "enough".

What are the basic properties of RNs? On any interval, there is an equal probability to obtain any value. For convenience, we will take this interval to be $(0,1)$. Note, we will generally exclude $0$ from the interval when considering RNs on a computer, since many algorithms will fail if $0$ is ever encountered.

At the very least, we expect our computer RNs to have the properties of abstract ones.

Mean:

$$\langle x \rangle = \int_0^1 \text{d}x\, x = \frac{1}{2}$$

Variance:

$$\langle x^2 \rangle - \langle x \rangle^2 = \frac{1}{3}-\frac{1}{4} = \frac{1}{12}$$

Since it is useful to check the mean and variance, let us write simple functions which calculate these values given an iterable object like a list or array.

In [ ]:
# Note, these functions are available in `numpy` as `numpy.mean` and
# `numpy.var`.
def mean(xs):
    """
    Return the mean for a list of numbers given by `xs`.

    xs: list of numbers to find the mean for.
    """
    return sum(vals) / float(len(vals))


def variance(vals):
    """
    Return the variance for a list of numbers given by `xs`.

    xs: list of numbers to find the variance for.
    """
    return mean([val**2 for val in vals]) - mean(vals) ** 2

In practice, computers represent only a finite range of numbers.
This has implications for RNGs. Any algorithm that is deterministic and shuffles through the numbers is bound to repeat itself. Further, if our algorithm uses one number as the seed for the next, this repetition will mean that the sequence of pRNs repeats itself. This hardly looks random. Much of the art in developing good RNGs is knowing how to characterize or calculate the length of these cycles. If the cycle is long enough, much longer than any sequence of RNs we ever use in practice, then there is hope that this sequence will have the desired properties of random numbers. Identifying an algorithm with a long cycle is a priority, and possibly led to the focus on modular arithmetic, to be discussed shortly.

## Middle square method

The need for such algorithms only came about with the development of practical computers and Monte Carlo (MC) techniques. Historically, we are placing ourselves in the 1940-1950s.
Von Neumann (who else?) used a method called the [middle square](https://en.wikipedia.org/wiki/Middle-square_method). It can give us some insight to the problem we are facing.

The middle square algorithm is short and strange:

1. input a number of length $n$
2. square the number and zero-pad if not of length $2n$
3. output the "middle" number of length $n$
4. use this number as the random variate and the input for step 1

For example:

$$14^2 = 196 \to 0196 \to 19; 19^2 = 0361 \to 36; 36^2 = 1296 \to 29; 29^2 = 0841 \to 84, \text{ etc.}$$

Von Neumann used larger $n$ than this, but you get the point.

Before we practically explore the middle square method, let us first consider some technical realities behind RNGs. Perhaps the most critical aspect is that all RNGs require state; for the next RN to be generated, the RNG must have saved information stored on how the previous RN was generated. In the context of Python, this means that we need to use a class to define an RNG.

In [ ]:
# Here, we define a simple base class which can hold the state for an RNG.
class RNG:
    """
    This is a simple base class for random number generation.
    """

    def __init__(self, seed):
        """
        Constructor for this class, e.g. `RNG(seed)`.

        seed: seed used to initialize the generator.
        """
        self.seed = seed
        # In derived classes, further state can be initialized here.

    def __call__(self):
        """
        Returns a uniform RN between 0 and 1. This defines the `()`
        for this class. For example:
        ```
        rng = RNG(seed)
        rng()
        ```
        """
        # In derived classes, the actual algorithm should be implemented here.
        return 0.5

We are also interested in the periodicity for the RNG, so we can define a function, which given an RNG, generates RNs until the sequence repeats.

In [ ]:
# Since it is useful to determine the period of an RNG, let us define a method
# that does just this. Since we are storing the RNs we need to be careful not
# to store a very long list.
def rng_sequence(rng, n=int(1e6)):
    """
    Given an RNG, return its sequence after its first repition or until
    `n` RNs have been generated.

    n: maximum number of RNs to generate.
    """
    # Start with no RN and create the RN sequence.
    rn, rns = None, []

    # Loop while the next RN is not in the sequence.
    while rn not in rns:
        # Append the RN to the sequence.
        if rn != None:
            rns.append(rn)
        # Generate the next RN.
        rn = rng()
        # Return if `n` is reached.
        if len(rns) >= n:
            return rns

    # Return the sequence.
    return rns

### Exercise: implement the middle square method

For $n$=2, try all seeds (inputs) and identify the longest sequence (before a repeat). For this longest sequence, calculate the mean and variance. Then, identify the worst seeds (0 period).

In [ ]:
# All RNGs require state of some sort, i.e. they need to save information about
# the previous random number call. This means that in Python, we should write
# our RNGs as classes.


# Here, we define the middle square generator as a derived class of `RNG`.
class RNGMiddleSquare(RNG):
    # For this example `n` would be 2.
    def __init__(self, seed, n):
        # This initializes the base class.
        super().__init__(seed)
        # Additional state initialized here.
        self.n = n

    def __call__(self):
        # Return the actual number.
        return 0


# Get the seed and create the RNG.
seed = int(input("Please enter a two-digit number:\n[##] "))
rng = RNGMiddleSquare(seed, 2)

# Determine the period.

## RNGs based on modular arithmetic

The middle square can have short periods (bad) and are not even very random (really bad).

These types of RNGs were quickly overshadowed by those based on modular arithmetic. Two numbers are of the same modular class if they differ by only multiples of an integer $m$. Thus,

$$1 \equiv 4\pmod{3} \equiv 7\pmod{3} \equiv 82\pmod{3}.$$

In Python, the mod operator is given by the `%` symbol. Modular arithmetic has some nice properties that allow us to calculate or estimate periods, and it can be done quickly on a computer (a feature that could be relevant when sampling millions of RNs). However, the equivalences can also lead to some unexpected consequences.

First, we will explore linear congruential generators. These have the form,

$$x_i = a x_{i-1} + c \mod m, 0 < x_i < m \in \mathbb{Z},$$

where $a$ is the *multiplier*, a positive integer, $m$ is the *modulus*, also an integer, and $c$ is the *increment*, a non-negative integer. As with all such algorithms, you can convert this into a value between $0$ and $1$ by dividing by the largest element $m$. In general, $a$ should be relatively prime to $m$. When $c \neq 0$ this type of RNG is called a linear congruential genenerator (LCG) and when $c = 0$ this RNG is called a multiplicative LCG (MLCG). In the examples below, we will focus on the special case of the MLCG.

In [ ]:
# We can now define a linear congruential generator.
class RNGLCG(RNG):
    # For our state, we only need the previously generated integer.
    # We give a default value of `c` so this is an MLCG.
    def __init__(self, seed, a=65593, m=2**31, c=0):
        super().__init__(seed)
        self.state = seed
        self.a = a
        self.m = m
        self.c = c

    # Here, we define the actual RNG.
    def __call__(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / (self.m - 1)

Now, let us test this RNG by first checking its period, mean, and variance.

In [ ]:
# Create the RNG.
rng = RNGLCG(9, 3, 31)

# Find the period.
rns = rng_sequence(rng)

# Print the period, mean, and variance.
print(f"period = {len(rns)}")
print(f"mean = {mean(rns)}")
print(f"variance = {variance(rns)}")

### Exercise: patterns in multiplicative congruential generator

Lets investigate some other properties of this RNG.
The pairs $(x_i,x_{i-1})$ lie on a plane. Plot their pattern.

In [ ]:
# We can use `matplotlib` to create a figure and axis.
fig, ax = plt.subplots()

# Use the `scatter` method of `ax` to plot the RNs.
xs = [0]
ys = [0]
ax.scatter(xs, ys)

### Exercise: choosing better parameters

A better choice of $a$ and $m$ can give our RNG better properties.
Create a new RNG with $a = 65539$ and $m = 2^{31}$. Generate a sequence of 1000 RNs using seed = $2^8 - 1$. Make a sequence of points $(x_i, x_{i-1})$ and plot them. Do any patterns emerge?

In [ ]:
# You already have the RNGLCG class, so use this, and then plot.

Now, given a little more advanced knowledge, let us see if we can find a pattern. Using the same RNG and seed, generate 20002 RNs and make a sequence of points $(x_{i-1}, x_i, x_{i+1})$. Filter these points for $0.50 < x_i < 0.51$. Make a 2-D scatter plot of $(x_{i-1},x_{i+1})$ for these filtered points. Do any patterns emerge?

So far, we have only been looking in two dimensions, what about in three? Generate 1000 RNs using the same RNG and seed. Then make a 3-D scatter plot with the points $(x_{i-1},x_i,x_{i+1})$.

In [ ]:
# The following will create a 3-D scatter plot.

# Set your points.
xs = [0]
ys = [0]
zs = [0]

# Create the figure and allow to be 3-D.
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Create the scatter plot.
ax.scatter3D(xs, ys, zs)

This example is not just pedagogical. In fact, this RNG is called `RANDU`. The authors of Numerical Recipes (3rd ed), share this anecdote:

> Even worse, many early generators happened to make particularly bad choices for m and a. One infamous such routine, `RANDU`, with a = 65539 and m = $2^{31}$, was widespread on IBM mainframe computers for many years, and widely copied onto other systems. One of us recalls as a graduate student producing a "random" plot with only 11 planes and being told by his computer center's programming consultant that he had misused the random number generator: "We guarantee that each number is random individually, but we don't guarantee that more than one of them is random." That set back our graduate education by at least a year!

## XORshift RNGs

Modern RNGs still combine two algorithms to remove undesired correlations.
However, they use an independent algorithm for the first sequence, so as not to unwittingly combine two correlations that arise from the same class of algorithm.

One such popular algorithm is called `XORshift`. Its properties are understood by studying the multiplication of 3 special kinds of binary matrices: the identity matrix $I$, a right-shift matrix $R$, and a left-shift matrix $L$. However, this is type of RNG is typically programmed more efficiently using bit-shift and exclusive OR (XOR) operations (true if exactly one of two inputs is true, false otherwise).

The resulting algorithm does not look anything like matrix multiplication, but it really is. This is because a bit shift can be represented on an $n$-bit vector (typically 32-bit or 64-bit) by a matrix with only ones on a sub-diagonal. Thus, to right-shift a bit sequence $\beta = (b_1,b_2,\cdots,b_n) \to \beta^\prime = (0,b_1,b_2,\cdots,b_{n-1})$, you would multiply $\beta$ by an $n\times n$ matrix $R$,

$$
\beta^\prime = \beta R
$$

with only $1$s above the diagonal.

$$
R \equiv
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0 \\
0 & 0 & 1 & \cdots & 0 \\
0 & 0 & 0 & 1 & 0 \\
\vdots & \vdots & \vdots & \ddots & 1 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$$

Similarly, a left-shift matrix has only $1$s on a subdiagonal below the diagonal.

$$
L =
\begin{pmatrix}
0 & 0 & 0 & \cdots & 0 \\
1 & 0 & 0 & \cdots & 0 \\
0 & 1 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & 0 \\
0 & 0 & 0 & 1 & 0 \\
\end{pmatrix}
$$

Finally, since these are binary matrices, all operations use integer arithmetic modulo 2, *i.e.* all arithmetic operations are defined as normal, but with the result taken as $\pmod 2$.

It is important to note here that the right-shift defined above is a *logical* right shift and not an *arithmetic* right shift. In a logical right shift, the left-most bit is set to zero, while for an arithmetic right-shift the left-most bit does not change. This is important for signed numbers, where the left-most bit gives the sign of the number. In most programming languages, the right-shift operator is defined as the arithmetic right shift and not the logical right shift.

The claim is that the series of operations $\beta T, \beta T^2 , \cdots \beta T^{2^n-1}$, every possible $\beta$ is produced. This means that the cycle has length $2^n-1$. For most current implementations, this transformation matrix can be defined as

$$
T = (1 + L^a)(1 + R^b)(1 + L^c)
$$

which is an $n\times n$ binary matrix with $(a,b,c)$ bit-shifts left-right-left. Here, $1$ is an $n\times n$ identity matrix. This identity matrix, in combination with the modulo 2 arithmetic, corresponds to an XOR operator. Only certain triplets of $(a,b,c)$ fulfill the ideal cycle length of $2^n - 1$. A minimal test that $T$ has the desired properties is to check that

$$
T^{2^n} = T
$$

holds.

### Exercise: implement bit shifting

Before we implement a full `XORshift` RNG, we need to build the necessary matrices to define the transformation $T$. Using the `sparse` sub-module from `scipy`, define functions that return $R$ and $L$ matrices. We use the `sparse` module to make the calculation more efficient than just full matrix multiplication.

In [ ]:
# The following creates a sparse identity matrix with n = 4.
im = sp.sparse.eye(4, dtype=np.int32)
# Here, the first argument is `n`, and the second argument is the
# data type of the matrix. All available `numpy` datatypes can be listed
# with the following.
print(np.sctypeDict)
# Depending on how we perform our modulo arithmetic, we could choose a
# more efficient data type for the matrix.

# It is possible to print a sparse matrix as an array to check its form.
print(im.toarray())

# We can also create off-diagonal matrices.
om = sp.sparse.diags([1], [1], shape=(4, 4), dtype=np.int32)
# The first and second arguments should be lists of the same length.
# The first argument is the value to insert along a diagonal of the matrix.
# The second argument is the offset for the diagonal.
# The `shape` argument gives the matrix dimensions.
# This example places 1s one above the main diagonal.
print(om.toarray())


# We then define the following two methods.
def build_rshift(n, dtype=np.int32):
    # Actually define the matrix here.
    return 0.0


def build_lshift(n, dtype=np.int32):
    # Actually define the matrix here.
    return 0.0


# For completeness define the identity matrix as well.
def build_i(n, dtype=np.int32):
    # Actually define the matrix here.
    return 0.0

In Python, the right-shift operator is given by `>>` followed the number of positions, and the left-shift operator is given by `<<`, also followed by the number of positions. Check that your shift matrices are correctly built using these built in operators, remembering that the right-shift operator is an arithmetic shift.

As a brief, but relevant aside, the Python `int` type does not have a fixed size and instead uses "arbitrary precision arithmetic" (also known as `bignum`). This means that you should use the fixed size-types from `numpy` instead. Note, real number types like `float` in Python are fixed size. Why is `float` fixed size but `int` variable size?

In [ ]:
# Be careful with what data type you check the operation on.
# Because we are working with logical shift operators,
# rather than arithmetic, this means you should work with
# unsigned types. The following creates an unsigned 8-bit
# integer.
val = np.uint8(120)

# You will need to be able to convert this value to binary.
# The following creates a string, where the characters
# are the binary representation of the decimal value,
# prefixed by "0b".
chars = bin(val)


# You can loop over this string to produce a vector of bits.
# It might be useful to write this out as a little function.
# Here, `dec` is the decimal value and `dtype` is the data type
# to encode the bits in the vector.
def dec_to_bits(dec, dtype=int):
    # The following determines the number of bytes used by `dec`.
    # Remember this in bytes and not bits!
    n = np.dtype(dec).itemsize
    # Define the conversion here.
    return []


# Now, shift the vector of bits.


# Once you have a shifted vector of bits, it is useful to be able to convert
# back into a decimal value. Here, `dtype` is the return type for the
# decimal value.
def bits_to_dec(bits, dtype=int):
    # First, create a binary string. Then, most numerical types in Python can
    # convert from strings for non-decimal values. The `2` indicates this is
    # a binary string.
    return dtype("01010", 2)

We can also consider the general form of $L^a$ and $R^b$. Is there a more efficient way to write $L^a$ then just `L**a`?

In [ ]:
# First, use matrix multiplication to see what L^a is for a from 1 to n.
# Next, see if there is a simpler way to construct these matrices.
# Finally, write new `build_rshift` and `build_shift` methods which allow the
# power to be specified.

### Exercise: determine suitable triplets

Construct $T$ for the triplets $(1,3,10)$, $(5,17,13)$, and $(2,5,14)$ using 32-bit precision. Which of these are suitable triplets, using the test $T^{2^n} = T$?

In [ ]:
# Define a method that constructs T.
# If the additional argument `reverse` is `True`, then perform a
# right-left-shift rather than left-right-left shift.
def build_tshift(n, a, b, c, reverse=False, dtype=np.int32):
    # Use the `build_rshift`, `build_lshift`, and `build_i` methods.
    return 0.0


# Test T^(2^n) = T. Remember the right-hand side should be modulo 2.
# With `scipy` it is possible to piecewise apply modulo 2 to a matrix
# but for a sparse matrix, it must be first promoted to a full matrix using
# `todense` as shown below.
s = 2 * sp.sparse.eye(4)
d = s.todense()
m = d % 2
print(f"m =\n{m}")

# Finally, the `numpy.count_nonzero` can be used to find the number of non-zero
# elements in an array (or matrix).
print(f"non-zero `d` elements = {np.count_nonzero(d)}")
print(f"non-zero `m` elements = {np.count_nonzero(m)}")

### Exercise: implement XORshift with matrices

Now that we have all the components, define an XORshift RNG. From the exercise above, choose valid default values for $a$, $b$, and $c$.

In [ ]:
# We can now define an XORshift generator using matrices.
class RNGXORShiftMatrix(RNG):
    # The `dtype` determines how many bits we use, and we should choose
    # a valid default for `a`, `b`, and `c`. If `reverse` is `True`,
    # reverse the direction of the shift operators. We need this later for a
    # combined RNG.
    def __init__(self, seed, a=1, b=1, c=1, reverse=False, dtype=np.uint32):
        super().__init__(seed)
        # Store the state and any other info that is needed.
        # Building the transformation matrix only once here would make sense.

    # Here, we define the actual RNG.
    def __call__(self):
        # Apply the necessary transformations here.
        return 0.5

### Exercise: implement XORshift with operators

It turns out, not suprisingly, that the `XORshift` RNG we implemented with matrices is not particularly efficient. How many operations per RNG call are required for the matrix implementation? Assume the matrices are dense and ignore the operations necessary to convert to from a vector of bits to a real number.

So, let us now implement a more efficient version of this generator where we use the built in Pythonic operators for XOR and bit shifting.

In [ ]:
# The following demonstrate the in-place XOR operator, `^=`.
a = np.uint8(13)
print(f"a   = {dec_to_bits(a)}")
b = np.uint8(5)
print(f"b   = {dec_to_bits(b)}")
a ^= b
print(f"a^b = {dec_to_bits(a)}")

# We already compared the bit-shifting operator, but we can illustrate it
# again here.
a = np.uint8(13)
print(f"a      = {dec_to_bits(a)}")
a = a << 1
print(f"a << 1 = {dec_to_bits(a)}")


# Here is a template for defining the generator.
class RNGXORShiftOperator(RNG):
    # Use the same default `a`, `b`, and `c` as RNGXORShiftMatrix.
    # Use the same default `dtype` as RNGXORShiftMatrix.
    def __init__(self, seed, a=1, b=1, c=1, reverse=False, dtype=np.uint32):
        super().__init__(seed)
        # Store the state.
        # Store the bit size.
        # Store the `a`, `b`, and `c` configuration.

    # Define the actual RNG call.
    def __call__(self):
        # Bit-shift `state` left by `a`, then take XOR with non-shifted state.
        # Bit-shift `state` right by `b`, then take XOR with non-shifted state.
        # Bit-shift `state` left by `c`, then take XOR with non-shifted state.
        # Normalize and return.
        return 0.5

Compare this `XORshift` RNG with the one you wrote using matrices. Do the two match? Should they?

In [ ]:
# Define the two RNGs. Make sure to use the same `seed` and `a`, `b`, and `c`.
# Loop over the first 5 RNs generated.

## Combined RNGs

We have explored three different RNG methods: [Middle Square Method](scrollTo=Middle_square_method), [LCGs](scrollTo=#RNGs_based_on_modular_arithmetic), and [`XORshift`](XORshift_RNGs). While only the final RNG method is currently considered to be sufficient, another possibility is to combine RNGs together to produce a better RNG. In Numerical Recipes (3rd ed), a number of guidelines are given for what constitutes a bad RNG, which is then followed by this "received wisdom of the present".

> An acceptable random generator must combine at least two (ideally, unrelated) methods. The methods combined should evolve independently and share no state. The combination should be by simple operations that do not produce results less random than their operands.

In Numerical Recipes, an RNG is recommended that combines four different RNGs. The first RNG is a an `XORshift`, the second is an LCG, the third is another `XORshift`, and the fourth is a Multiply with Carry (MWC). These RNGs are combined as follows.

$$
(\text{XOR}_1(\text{LCG}_2) + \text{XOR}_3) \wedge \text{MWC}_4
$$

The notation here of $\text{XOR}_1(\text{LCG}_2)$ is called a succesor relation, where the state of $\text{LCG}_2$ is still iterated as normal, but is taken as the state of $\text{XOR}_1$.

### Exercise: implement an MWC RNG

The idea behind an MCW is similar to an LCG with $m = ab - 1$, but is implemented using a bit-shift with an AND operation. The algorithm is as follows, considering an $n$-bit state.
1. Right-shift $x_i$ by $n/2$.
2. Take the AND operation between $x_i$ and $2^{n/2} -1$.
2. Multiply the result of (2) by $a$ and add to the result of (1).
Implement an MCW for $n = 64$ and $a = 4294957665$.

In [ ]:
# Class to perform MWC.
class RNGMWC(RNG):
    def __init__(self, seed, a=4294957665, dtype=np.uint64):
        super().__init__(seed)
        # Store the state.
        # Store the bit size.
        # Store the `a` configuration.

    # Define the actual RNG call.
    def __call__(self):
        # Bit-shift `state` right by `n/2`.
        # Take bit-wise AND with 2^(`n`/2) - 1, multiply by `a`.
        # Here, you will need to create a dtype of 2^(`n`/2) otherwise Python
        # will complain about adding an `int` to dtype.
        # Add the results.
        # Normalize and return.
        pass

### Exercise: combine RNGs

We now have all the ingredients needed to create the recommended RNG combination from Numerical recipes. However, we need to know the parameters used for each RNG.

* $\text{XOR}_1$: $a = 21$, $b = 35$, $c = 4$, and left-right-left shifts
* $\text{LCG}_2$: $a = 2862933555777941757$, $c = 7046029254386353087$, and $m = 2^n$
* $\text{XOR}_3$: $a = 17$, $b = 31$, $c = 8$, and right-left-right shifts
* $\text{MWC}_4$: $a = 4294957665$

For all the generators we have $n = 64$. There is also some additional initialization that is required that is described in the comments for the exercise code.

To check your solution, the first $5$ random numbers should be the following.
```
0.40573455184711105
0.4626566077784852
0.06435855239385686
0.5209198360653725
0.034725647607092826
```

In [ ]:
# Class to create a combined RNG.
class RNGCombine(RNG):
    def __init__(
        self,
        seed,
        seed3=4101842887655102017,
        seed4=1,
        a1=21,
        b1=35,
        c1=4,
        a2=2862933555777941757,
        c2=7046029254386353087,
        m2=2**64,
        a3=17,
        b3=31,
        c3=8,
        a4=4294957665,
        dtype=np.uint64,
    ):
        super().__init__(seed)
        # Determine the seeds for the RNGs. Any value for `seed1` is fine,
        # since this seed is never used in RNG1.
        # The seed for RNG2 is the XOR of `seed1` and `seed3`.
        # We already have `seed3` and `seed4`.

        # Store the bit size and data type.

        # Create the 4 RNGs.

        # Perform some additional initialization.
        # Call the RNG with the current states.
        # Update the state of RNG3 to that of RNG2.
        # Call the RNG with the current states.
        # Update the state of RNG4 to that of RNG3.
        # Call the RNG with the current states.

    # Define the actual RNG call.
    def __call__(self):
        # Independently call RNG2, RNG3, and RNG4.
        # Set the RNG1 state to the RNG2 state. Make sure this is the correct
        # data type.
        # Call RNG1.
        # Combine the states.
        # This step can cause an overflow error, but it is safe to ignore
        # since the overflow treatment is what we want.
        # Normalize and return.
        return 0.5

## Tests of Random Number Generators

At the begining of this notebook, we defined a good RNG based on its mean and variance matching an expected uniform distribution. However, as we saw with various RNGs, the period of a generator can be small, so it begins repeating, or patterns can even arise within a single period. However, as we saw for LCGs that even if we [chose better parameters](#scrollTo=Exercise:_choosing_better_parameters) these patterns could continue to emerge. So, how do we catch these patterns, and what defines a good RNG?

There is not a simple answer to this question, but instead there are standard series of tests that can be applied to RNGs that attempt to quantify how good an RNG is. Perhaps the best known is the `diehard` test suite developed by George Marsaglia which was extended by Robert Brown into the [`dieharder`](https://webhome.phy.duke.edu/~rgb/General/dieharder.php) test suite, and by Pierre L'Ecuyer and Richard Simard into the [`TestU01`](https://github.com/umontreal-simul/TestU01-2009/) test suite. We will explore a few tests from the original `diehard` suite of tests.

### Exercise: timing

Timing does not assess the quality of an RNG, but is necessary from a practical point of view to determine whether an RNG is usable. Evaluate the timing per RNG call for all the RNGs implemented in this notebook.

In [ ]:
# Define a list of RNG classes.
rng_classes = [RNGCombine]

# Create a list of RNGs.
rngs = [rng_class(13) for rng_class in rng_classes]
# Time each class.

# The following times everything between start and top.
# You will probably want to make a number of RNG calls to
# get an average timing.
import time

start = time.time()
for rng in rngs:
    rng()
stop = time.time()
diff = stop - start

### Exercise: Kolmogorov–Smirnov test

The general idea behind the `diehard` tests is to generate a distribution produced from an algorithm using the RNG and compare this to the asymptotic limit of the distribution for the algorithm. To do this, we need to be able to quantify how well a discrete generated distribution agrees with a continuous distribution. The Kolmogorov-Smirnov (KS) test does this by finding the maximum distance between the cumulative distribution functions of the two distributions. Try to implement this below.

Now, let us test a uniform distribution with the KS test.

However, this number is not so useful unless we know the expected distribution for KS distances. The asymptotic CDF for the distance $d$ as $n \to \infty$ for the generated distribution is given by the following.

$$
\text{CDF}(d) = \frac{\sqrt{2\pi}}{\sqrt{n}d} \sum_{k = 1}^\infty e^{-(2k-1)^2\pi^2/(8nd^2)}
$$

Implement a function which calculates the p-value for the KS test.

In [ ]:
# Define the function here, where `kmax` defines the upper limit of the sum.
def ks_pvalue(d, n, kmax=100):
    return 0.0

We can test our implementation against the `scipy` implementation.

In [ ]:
p = ks_pvalue(d, len(xs))
s = sp.stats.kstest(xs, cdf_uniform)
print(f"local distance: {d}")
print(f"scipy distance: {s.statistic}")
print(f"local distance: {p}")
print(f"scipy distance: {s.pvalue}")

### Exercise: minimum distance test

There are quite a few tests in the `diehard` including tests such as the birthdays, craps, car parking, and monkeys. We will focus on a test with a slightly more boring name, the minimum distance test. In this test, $n$ random points $(x_i, y_i)$ are generated where both $x_i$ and $y_i$ are uniformly distributed between $0$ and $10^4$. The minimum distance between all point combinations, $d_\min^2$, is then returned. Implement this test below.

In [ ]:
# Define the method.
def diehard_minimum_distance(rng, n):
    # Create the points.
    # Make sure they are sampled between 0 and 1e4.
    # Calculate the distance.
    # Return the minimum distance.
    return 1e8

This value itself, or an ensemble of these values, is not useful unless we know the corresponding distribution. Create a distribution from $100$ $d_\min^2$ using $n = 8000$ and the `RNGCombine` RNG. **This will take some time.**

In [ ]:
# Create the RNG.
# Run the test 100 times with 8000 points each.

Plot the histogram for these $d_\min^2$. What kind of distribution does this look like?

In [ ]:
# Create the figure.
# Use the `ax.hist` to create a histogram.

It turns out the distribution is an exponential.

$$
\text{PDF}(x) = \mu e^{-\mu x}
$$

The CDF is then given by the following.

$$
\text{CDF}(x) = 1 - e^{-\mu x}
$$

For this particular test, $\mu = 1/0.995$. Implement the expected CDF for this test and calculate the KS p-value.

In [ ]:
# First, we define the CDF.
def cdf_exp(x, mu=1 / 0.995):
    return 0.0


# Now we perform the test.

We start to get worried if the p-value is close to $0$ or $1$. Normally, something less than $0.05$ would be considered problematic. However, there are a sufficient number of `diehard` tests that failures at the $0.05$ level can still occur for an RNG that is ideal. However, when p-values are less than $1\times10^{-5}$ this is typically indicative of an RNG that has some serious shortcomings.